In [1]:
import requests
from time import sleep 
import numpy as np
from keras.callbacks import EarlyStopping
import random
user_id= '0SIMZCTG7KZACJPQB7FU'
base_url='http://35.180.178.243'
url_reset=base_url+'/reset'
url_predict=base_url+'/predict'
params={'user_id':user_id}
data= requests.get(url_reset, params=params).json()
nb_users=data['nb_users']
nb_items=data['nb_items']
state_history=data['state_history']
rewards_history=data['rewards_history']
next_state=data['next_state']
ref_state=next_state
action_history=data['action_history']

#Distinguish positive item from negative item and return them 
def sample_triplets():
    user_ids=list()
    pos_item_ids=list()
    neg_item_ids=list()
    for i in range(200):
        if rewards_history[i]>0:
            user_ids.append(state_history[i][action_history[i]][0])
            pos_item_ids.append(state_history[i][action_history[i]][1])
            r=list(range(1,state_history[i][action_history[i]][1]))+list(range(state_history[i][action_history[i]][1]+1,nb_items))
            neg_item_ids.append(random.choice(r))
    return [user_ids, pos_item_ids, neg_item_ids]

C:\Users\RAMZI\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf


def identity_loss(y_true, y_pred):
    """Ignore y_true and return the mean of y_pred
    
    This is a hack to work-around the design of the Keras API that is
    not really suited to train networks with a triplet loss by default.
    """
    return tf.reduce_mean(y_pred + 0 * y_true)


def margin_comparator_loss(inputs, margin=1.):
    """Comparator loss for a pair of precomputed similarities
    
    If the inputs are cosine similarities, they each have range in
    (-1, 1), therefore their difference have range in (-2, 2). Using
    a margin of 1. can therefore make sense.

    If the input similarities are not normalized, it can be beneficial
    to use larger values for the margin of the comparator loss.
    """
    positive_pair_sim, negative_pair_sim = inputs
    return tf.maximum(negative_pair_sim - positive_pair_sim + margin, 0)

In [3]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, Dense, Dropout
from keras.layers import Concatenate, Lambda
from keras.regularizers import l2


def make_interaction_mlp(input_dim, n_hidden=1, hidden_size=64,
                         dropout=0, l2_reg=None):
    """Build the shared multi layer perceptron"""
    mlp = Sequential()
    if n_hidden == 0:
        # Plug the output unit directly: this is a simple
        # linear regression model. Not dropout required.
        mlp.add(Dense(1, input_dim=input_dim,
                      activation='relu', kernel_regularizer=l2_reg))
    else:
        mlp.add(Dense(hidden_size, input_dim=input_dim,
                      activation='relu', kernel_regularizer=l2_reg))
        mlp.add(Dropout(dropout))
        for i in range(n_hidden - 1):
            mlp.add(Dense(hidden_size, activation='relu',
                          W_regularizer=l2_reg))
            mlp.add(Dropout(dropout))
        mlp.add(Dense(1, activation='relu', kernel_regularizer=l2_reg))
    return mlp


def build_models(n_users, n_items, user_dim=32, item_dim=64,
                 n_hidden=1, hidden_size=64, dropout=0, l2_reg=0):
    """Build models to train a deep triplet network"""
    user_input = Input((1,), name='user_input')
    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')

    l2_reg = None if l2_reg == 0 else l2(l2_reg)
    user_layer = Embedding(n_users, user_dim, input_length=1,
                           name='user_embedding', embeddings_regularizer=l2_reg)

    # The following embedding parameters will be shared to encode both
    # the positive and negative items.
    item_layer = Embedding(n_items, item_dim, input_length=1,
                           name="item_embedding", embeddings_regularizer=l2_reg)

    user_embedding = Flatten()(user_layer(user_input))
    positive_item_embedding = Flatten()(item_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_layer(negative_item_input))


    # Similarity computation between embeddings using a MLP similarity
    positive_embeddings_pair = Concatenate(name="positive_embeddings_pair")(
        [user_embedding, positive_item_embedding])
    positive_embeddings_pair = Dropout(dropout)(positive_embeddings_pair)
    negative_embeddings_pair = Concatenate(name="negative_embeddings_pair")(
        [user_embedding, negative_item_embedding])
    negative_embeddings_pair = Dropout(dropout)(negative_embeddings_pair)

    # Instanciate the shared similarity architecture
    interaction_layers = make_interaction_mlp(
        user_dim + item_dim, n_hidden=n_hidden, hidden_size=hidden_size,
        dropout=dropout, l2_reg=l2_reg)

    positive_similarity = interaction_layers(positive_embeddings_pair)
    negative_similarity = interaction_layers(negative_embeddings_pair)

    # The triplet network model, only used for training
    triplet_loss = Lambda(margin_comparator_loss, output_shape=(1,),
                          name='comparator_loss')(
        [positive_similarity, negative_similarity])

    deep_triplet_model = Model(inputs=[user_input,
                                       positive_item_input,
                                       negative_item_input],
                               outputs=[triplet_loss])

    # The match-score model, only used at inference
    deep_match_model = Model(inputs=[user_input, positive_item_input],
                             outputs=[positive_similarity])

    return deep_match_model, deep_triplet_model


hyper_parameters = dict(
    user_dim=32,
    item_dim=64,
    n_hidden=1,
    hidden_size=128,
    dropout=0.1,
    l2_reg=0
)
deep_match_model, deep_triplet_model = build_models(nb_users, nb_items,
                                                    **hyper_parameters)


deep_triplet_model.compile(loss=identity_loss, optimizer='adam')
fake_y = np.ones_like(sample_triplets()[0])

# Sample new negatives to build different triplets at each epoch
triplet_inputs = sample_triplets()

# Fit the model incrementally by doing a single pass over the
# sampled triplets.
deep_triplet_model.fit(triplet_inputs, fake_y, shuffle=True,
                       batch_size=64, epochs=30)
reward=0
expected=0
for i in range(1000):
    items_to_rank=list()
    for k in  range(len(next_state)):
        items_to_rank.append(next_state[k][1])
    pos_reward=np.argmax(deep_match_model.predict([(np.ones((1,len((items_to_rank))))*next_state[0][0])[0], np.array(list(items_to_rank))], batch_size=32))
    pos_state=next_state[pos_reward][2]
    params['recommended_item']=pos_reward
    d=requests.get(url_predict, params=params).json()
    #print(f'expected:{d["reward"]},item_id:{next_state[pos_reward][1]},prediction:{next_state[pos_reward][2]}')
    reward+=d["reward"]
    #expected+=d["reward"]
    next_state=d["state"]     
#print(expected)
print('\n')
print(reward)

Epoch 1/30
54/54 [==============================] - 0s 6ms/step - loss: 1.0013
Epoch 2/30
54/54 [==============================] - 0s 37us/step - loss: 0.9871
Epoch 3/30
54/54 [==============================] - 0s 37us/step - loss: 0.9831
Epoch 4/30
54/54 [==============================] - 0s 37us/step - loss: 0.9738
Epoch 5/30
54/54 [==============================] - 0s 37us/step - loss: 0.9604
Epoch 6/30
54/54 [==============================] - 0s 37us/step - loss: 0.9568
Epoch 7/30
54/54 [==============================] - 0s 37us/step - loss: 0.9463
Epoch 8/30
54/54 [==============================] - 0s 37us/step - loss: 0.9333
Epoch 9/30
54/54 [==============================] - 0s 37us/step - loss: 0.9207
Epoch 10/30
54/54 [==============================] - 0s 46us/step - loss: 0.9163
Epoch 11/30
54/54 [==============================] - 0s 37us/step - loss: 0.9011
Epoch 12/30
54/54 [==============================] - 0s 37us/step - loss: 0.8894
Epoch 13/30
54/54 [===================

In [4]:
## Native algorithme which take the highest price and recommand this item 
## We did this to make a refence to our recommender system

In [5]:
def recom_item_pos(next_state):
    reward=list()
    for i in range(len(next_state)):
        reward.append(next_state[i][2])
    return np.argmax(reward) 

In [6]:
reward=0
next_state=ref_state
for i in range(1000):
    params['recommended_item']=recom_item_pos(next_state)
    d=requests.get(url_predict, params=params).json()
    reward+=d["reward"]
    next_state=d["state"]
print(reward)

232545.67882299318
